In [62]:
class table_order():
    def __init__(self) -> None:
        self.prepare()
        #self.order_day()
        #self.order_number()
        self.set_order_itens()

    def prepare(self):
        from A01_TOOL_BOX import tool_box
        tool_box=tool_box()

        self.pandas=tool_box.pandas
        self.random=tool_box.random
        self.datetime=tool_box.datetime

        self.sales_orders=self.pandas.read_csv(filepath_or_buffer='./NEW_DATASET/A03_TAB_CLIENTS.csv',
                                               sep=';',decimal=',')[['ID_client','cluster','visit_day','ID_sector']]
        
    def order_day(self):
        #to keep the problem simple, every client can make only at his visit day
        #get a set of dates
        import datetime
        dt_start=datetime.date(year=2023,month=1,day=2)
        days=datetime.date(year=2023,month=4,day=30)-dt_start
        days=days.days        
        dt_list=[dt_start + datetime.timedelta(days=i) for i in range(days)]
        dt_list=[i for i  in dt_list]
        dt_list=self.pandas.DataFrame(dt_list,columns=['Date'])
        dt_list['week']=dt_list['Date'].map(lambda x:x.isocalendar().week)
        dt_list['weekDay']=dt_list['Date'].map(lambda x:x.isocalendar().weekday)
        dt_list['month']=dt_list['Date'].map(lambda x:x.month)
        #set the date to the client
        self.sales_orders=self.pandas.merge(left=self.sales_orders,right=dt_list,left_on='visit_day',right_on='weekDay',how='left')
        self.sales_orders.drop(columns=['weekDay','visit_day'],inplace=True)
        #set the chance the client can by nothing at month
        self.sales_orders.loc[:,'filter']=''
        for i in self.sales_orders['ID_client'].unique():
            for j in self.sales_orders['month'].unique():
                if self.random.random()<0.05: # 5% of chance a client will not buy any product at the month
                    filter=(self.sales_orders['ID_client']==i) & (self.sales_orders['month']==j)
                    self.sales_orders.loc[filter,'filter']='remove'
            if self.random.random()<0.10: #10% of chance a client will not buy at the visit
                filter=(self.sales_orders['ID_client']==i) 
                self.sales_orders.loc[filter,'filter']='remove'
        #remove the filter orders
        self.sales_orders=self.sales_orders[self.sales_orders['filter']!='remove'].reset_index(drop=True)
        self.sales_orders.drop(columns=['filter'],inplace=True)
    
    def order_number(self):
        #set an ID of the order to register 
        self.sales_orders['order_n']=0
        self.sales_orders.sort_values(by=['Date','ID_client'],inplace=True)
        self.sales_orders.reset_index(inplace=True,drop=True)
        new=1
        self.sales_orders.loc[0,'order_n']=1
        for i in self.sales_orders.index[1:]:
            if self.sales_orders.loc[i,'Date']==self.sales_orders.loc[i-1,'Date']:
                new=self.sales_orders.loc[i-1,'order_n']+1
            else:
                new=1
            self.sales_orders.loc[i,'order_n']=new
        self.sales_orders['order_n']=self.sales_orders['Date'].map(lambda x:str(x).replace('-','').zfill(4))+self.sales_orders['order_n'].map(lambda x:str(x))
    
    def set_order_itens(self):
        #use random factors to set volume and itens in orders
        #create random factors to use as base
        def to_df(dictionary):
            dictionary=[[i,dictionary[i]] for i in dictionary ]
            dictionary=self.pandas.DataFrame(dictionary,columns=['cluster','factor'])
            return dictionary
        
        ref_cluste_volume={ #set how big a sale
            'A':[1,15],
            'B':[1,30],
            'C':[35,75],
            'D':[50,100]
        }
        ref_cluste_volume=to_df(ref_cluste_volume).rename(columns={'factor':'volume_factor'}).set_index('cluster')
        self.sales_orders=self.pandas.merge(left=self.sales_orders,right=ref_cluste_volume,left_on='cluster',right_index=True,how='left')
        self.sales_orders['volume_factor']=self.sales_orders['volume_factor'].map(lambda x: self.random.randint(x[0],x[1]))

        ref_cluster_products={ #set how many kinds of products
            'A':[1,7],
            'B':[2,10],
            'C':[7,17],
            'D':[10,20]
        }
        ref_cluster_products=to_df(ref_cluster_products).rename(columns={'factor':'products_factor'}).set_index('cluster')
        self.sales_orders=self.pandas.merge(left=self.sales_orders,right=ref_cluster_products,left_on='cluster',right_index=True,how='left')
        self.sales_orders['products_factor']=self.sales_orders['products_factor'].map(lambda x:self.random.randint(x[0],x[1]))
        #set the itens 
        ref_products=self.pandas.read_csv('./NEW_DATASET/A01_TAB_PRODUCTS.csv',sep=';',decimal=',')
        display(ref_products)
        itens_table=[]
        for i in self.sales_orders['order_n']:
            choose=self.random.sample(ref_products['ID_products'],self.)
            ref_products['ID_products']
            pass

     
        display(self.sales_orders)
table_order()

,ID_products,Product,cluster,units/pack,cost,markup,Target_price
0,901000,20in Monitor,Monitor,13,90.50,0.20,108.60
1,901001,27in 4K Gaming Monitor,Monitor,18,323.84,0.20,388.61
2,901002,27in FHD Monitor,Monitor,24,124.48,0.20,149.38
3,901003,34in Ultrawide Monitor,Monitor,21,315.20,0.20,378.24
4,902004,AA Batteries (4-pack),Others,94,1.41,1.00,2.81
5,902005,AAA Batteries (4-pack),Others,97,1.01,1.00,2.03
6,902006,Apple Airpods Headphones,Others,50,74.51,1.00,149.02
7,902007,Bose SoundSport Headphones,Others,42,49.69,1.00,99.37
8,901008,Flatscreen TV,Monitor,15,249.46,0.20,299.35
9,903009,Google Phone,Phone,19,461.54,0.30,600.00


,ID_client,cluster,visit_day,ID_sector,volume_factor,products_factor
0,10001-001,A,1.0,201,12,2
1,10001-002,C,2.0,201,65,14
2,10001-003,C,3.0,201,74,17
3,10001-004,D,4.0,201,92,11
4,10001-005,C,5.0,201,41,8
...,...,...,...,...,...,...
665,98101-069,B,6.0,110,26,8
666,98101-070,B,7.0,110,29,2
667,98101-071,C,1.0,110,59,11
668,98101-072,D,2.0,110,83,10
